In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import explode, col, count, collect_list, array,lower
from pyspark.sql.window import Window
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Assignment").getOrCreate()

In [0]:
#dbutils.fs.mv("dbfs:/FileStore/tables/skill2vec50K.csv.gz", "dbfs:/FileStore/tables/skill2vec_50K.csv.gz")
#dbutils.fs.mv("dbfs:/FileStore/tables/Technology_Skills.txt", "dbfs:/FileStore/tables/Technology_Skills.txt")
#df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/mafernandes1@sheffield.ac.uk/skill2vec_1K.csv")
#df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/mafernandes1@sheffield.ac.uk/skill2vec_10K.csv")
#dbutils.fs.mv("dbfs:/FileStore/shared_uploads/mafernandes1@sheffield.ac.uk/skill2vec_1K.csv", "dbfs:/FileStore/tables/skill2vec_1K.csv")
#dbutils.fs.mv("dbfs:/FileStore/shared_uploads/mafernandes1@sheffield.ac.uk/skill2vec_10K.csv", "dbfs:/FileStore/tables/skill2vec_10K.csv")

In [0]:
#Read file skill2vec_50K_csv.gz
filepath="dbfs:/FileStore/tables/"
filename_skills= "skill2vec_50K.csv.gz"

num_cols = 1000  # Set the maximum number of columns (961)

#Dynamically define the Schema
schema = StructType([StructField("JD_id", StringType(), True)] + [StructField(f"skill_{i}", StringType(), True) for i in range(1, num_cols)])

#Read the csv file

df_skills = spark.read.option("header", False).option("sep", ",").schema(schema).csv(filepath+filename_skills) 

#Convert the long dataframe where each column corresponds to a particular skill to each row corresponding to a skill

df_skills = df_skills.select("JD_id", explode(array([col(f"skill_{i}") for i in range(1, num_cols)])).alias("skill"))

In [0]:
#Read file Technology_Skills

filename_tech="Technology_Skills.txt"
df_tech_skills = spark.read \
    .option("header", True) \
    .option("delimiter", "\t") \
    .option("inferSchema", True) \
    .option("mode", "DROPMALFORMED") \
    .csv(filepath+filename_tech)


In [0]:
#Preprocessing of the file skill2vec_50K_csv.gz

#Remove unnecessary rows that are null  
df_skills = df_skills.dropna(subset = "skill")

#Remove Duplicates
df_skills = df_skills.dropDuplicates()

#Remove trailing and leading spaces 
#df_skills=df_skills.select("JD_id").apply(lambda x: x.strip())

#Assign the data types
df_skills = df_skills.withColumn('JD_id', df_skills['JD_id'].cast('int'))
df_skills = df_skills.withColumn('skill', df_skills['skill'].cast('string'))



In [0]:
#Preprocessing of the file Technology_Skills


#Specify the datatypes
df_tech_skills = df_tech_skills.withColumn('O*NET-SOC Code', df_tech_skills['O*NET-SOC Code'].cast('string'))
df_tech_skills = df_tech_skills.withColumn('Example', df_tech_skills['Example'].cast('string'))
df_tech_skills = df_tech_skills.withColumn('Commodity Code', df_tech_skills['Commodity Code'].cast('int'))
df_tech_skills = df_tech_skills.withColumn('Commodity Title', df_tech_skills['Commodity Title'].cast('string'))
df_tech_skills = df_tech_skills.withColumn('Hot Technology', df_tech_skills['Hot Technology'].cast('string'))
df_tech_skills = df_tech_skills.withColumn('In Demand', df_tech_skills['In Demand'].cast('string'))

#Rename Columns to appropriate names
df_tech_skills = df_tech_skills.withColumnRenamed('O*NET-SOC Code','Code')
#Rename Columns to appropriate names
df_tech_skills = df_tech_skills.withColumnRenamed('Example','Skill')

#Convert skills column to lower case 
df_tech_skills=df_tech_skills.withColumn("Skill", lower(col("Skill")))

In [0]:
#Q1 Number of job descriptions
job_description_count = df_skills.select("JD_id").distinct().count()
job_description_count

Out[27]: 50000

In [0]:
#Q2 Frequencies with which distinct skills are mentioned in JD and top 10 in desc order

top_skills=df_skills.groupBy("skill").agg(count("*").alias("Count")).sort(col("count").desc()).limit(10).show()

+--------------------+-----+
|               skill|Count|
+--------------------+-----+
|                Java| 1911|
|          Javascript| 1770|
|               Sales| 1705|
|Business Development| 1545|
|    Web Technologies| 1313|
|Communication Skills| 1305|
|         development| 1238|
|           Marketing| 1184|
|             Finance| 1078|
|                HTML| 1067|
+--------------------+-----+



In [0]:
#Q3 Five most frequent number of skills in the dataset

df_jd_skill_counts = df_skills.groupBy("JD_id").agg(count("skill").alias("count_skill"))
df_skill_freq = df_jd_skill_counts.groupBy("count_skill").count().orderBy("count", ascending=False).limit(5)

# Rename the columns
df_skill_freq = df_skill_freq.withColumnRenamed("count_skill", "num skills").withColumnRenamed("count", "Freq")

# Show the result
df_skill_freq.show()

+----------+-----+
|num skills| Freq|
+----------+-----+
|        10|10477|
|         5| 3432|
|         6| 3405|
|         1| 3386|
|         7| 3345|
+----------+-----+



In [0]:
#Q4Frequencies with which distinct skills are mentioned in JD and top 10 in desc order (Case insesitive)

top_skills_caseinsensitive=df_skills.withColumn("skill", lower(col("skill"))).groupBy("skill").agg(count("*").alias("Count")).sort(col("count").desc()).limit(10).show()

+--------------------+-----+
|               skill|Count|
+--------------------+-----+
|                java| 2759|
|          javascript| 2738|
|               sales| 2680|
|business development| 2108|
|           marketing| 1809|
|                 sql| 1564|
|              jquery| 1547|
|                html| 1539|
|communication skills| 1537|
|                 bpo| 1530|
+--------------------+-----+



In [0]:
#Q5
df_joined = df_tech_skills.alias("ts").join(df_skills.alias("s"), lower(df_tech_skills['skill']) == lower(df_skills['skill']), how='inner')


join_count = df_joined.select("ts.skill").count()
tech_skills_count = df_tech_skills.select("skill").count()
jd_skill_count = df_skills.select("skill").count()
join_count = df_joined.select("ts.skill").count()
print("Before Join",jd_skill_count )
#print("Number of skills in Tech dataset: ",tech_skills_count )
print("After Join",join_count )

Before Join 463803
After Join 1101498


In [0]:
#Q6
df_count=df_joined.groupBy("Commodity Title").agg(count("*").alias("Count")).sort(col("count").desc()).limit(10).show()

+--------------------+------+
|     Commodity Title| Count|
+--------------------+------+
|Object or compone...|324521|
|Web platform deve...|298754|
|Operating system ...|190926|
|Development envir...| 53013|
|Data base managem...| 44132|
|Analytical or sci...| 33552|
|Web page creation...| 31682|
|Data base user in...| 29436|
|Spreadsheet software| 18568|
|File versioning s...| 13846|
+--------------------+------+

